## Exploration

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["الجزائر في حالة جيدة", "أشعر بنعاس شديد"]

model = SentenceTransformer('medmediani/Arabic-KW-Mdel', device='cuda')
embeddings = model.encode(sentences)
print(embeddings)

In [4]:
import pandas as pd

df = pd.read_csv('data/data.csv')
df.rename({'bait_clean': 'البيت'}, axis=1, inplace=True)
df.head()

,id,البيت
0,1,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وتنتظ...
1,2,فما لبث يوما بسابق مغنم ولا سرعتي يوما بسابقة ...
2,3,وإن تنظراني اليوم أقض لبانة وتستوجبا منا علي و...
3,4,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,5,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا وأصعدا


In [5]:
embeddings = model.encode(df['البيت'])
embeddings.shape

(1831778, 768)

## Vector DB

In [7]:
import numpy as np
import faiss

# Number of vectors
num_vectors = embeddings.shape[0]
# Dimension of each vector
dimension = embeddings.shape[1]

# Initialize the index
index = faiss.IndexFlatL2(dimension)

# Add vectors to the index
index.add(embeddings)

# Print the number of vectors in the index
print(f'Total vectors in the index: {index.ntotal}')

Total vectors in the index: 1831778


In [8]:
faiss.write_index(index, 'vector_index.faiss')

## Old Code

In [9]:
verses = df.sample(50)['البيت'].values
verses

array(['وكان عقيق دمعي من جفاه    من العينين يتبع شبه عين',
       'خشنت على الأعداء في الحرب ملمسا    وغصنك غض في الشبيبة شارخ',
       'فدع عنك تطواع الهوى واتباعه    وشمر فقد فاز المجد المشمر',
       'يا من أراه للزمان حسنه    ومن حوى من كل علم حسنه',
       'هكذا يدفق يا نا    عس في حسنك حسني',
       'كأن الفاتحين من الأوالي    على أسيافهم رفعوا خيامه',
       'وقفنا ورسل الشوق بيني وبينها    حواجب أدت بثنا وأصابع',
       'هم لعمري أذل من قدم النع    ل نفوسا والنعل منهم أجل',
       'فإنا وجدنا فى الكتاب المقدم    فضيلته تعلو على فضل آدم',
       'وما زال حتى قلت لا بد أنه    مسامي الوحيد وازدهته الجرائم',
       'قد وفت المزن بميعادها    وخصت الروض بإسعادها',
       'فما غاية نرجو فتهدأ روعة    وفي كل بدء للأمور ختام',
       'ألم يتل النبي عليك قولا    من القرآن ذكرا يا كفورا',
       'كي لا يرى أثر بخدي أو يش    م شذاي منك فيظهر الأمران',
       'فما لياليك والأيام شاهدة    إلا تواريخ خير تكتب العجبا',
       'كأن من بطنانها ظهرانها    تحدر سيلا عرما للسائل',
       'الصبا وا

In [43]:
from scipy import spatial
import numpy as np

query = ['انقضاء الهوى']
query_embd = model.encode(query)[0]

def calc_similarity(arr):
    return spatial.distance.cosine(query_embd, arr)

embeddings = np.array(model.encode(verses))
similarities = np.apply_along_axis(calc_similarity, 1, embeddings)
similarities

array([0.4846797 , 0.6247592 , 0.52922347, 0.56900987, 0.45936608,
       0.5907397 , 0.42532855, 0.62579292, 0.70532355, 0.43667799,
       0.54305774, 0.4712894 , 0.79588178, 0.53239825, 0.49879676,
       0.54809141, 0.34138894, 0.55566293, 0.59161118, 0.62311333,
       0.61304009, 0.54524085, 0.45934892, 0.5529955 , 0.58100671,
       0.58447826, 0.39621145, 0.51740962, 0.66613737, 0.54452011,
       0.61856243, 0.59821153, 0.71583551, 0.70331854, 0.70902526,
       0.46037364, 0.59089184, 0.55707338, 0.57946232, 0.47321135,
       0.59717503, 0.58997414, 0.49016517, 0.61277238, 0.57614392,
       0.577066  , 0.71826899, 0.48471534, 0.55112913, 0.41280955])

In [44]:
verses[similarities.argsort()]

array(['الصبا والهوى وبيض الأماني    ذهبت كلها كأمس الذاهب',
       'يبددان من الأحزان ما احتشدا    ',
       'فثب وثبة فيها المنايا أو المنى    فكل محب للحياة ذليل',
       'وقفنا ورسل الشوق بيني وبينها    حواجب أدت بثنا وأصابع',
       'وما زال حتى قلت لا بد أنه    مسامي الوحيد وازدهته الجرائم',
       'لعمري ما الفرسان القوافي    لحاق ان ذاك من المحال',
       'هكذا يدفق يا نا    عس في حسنك حسني',
       'ولكن نأيت اليوم عنا فأصبحت    جوانحنا من حزنها تتفطر',
       'فما غاية نرجو فتهدأ روعة    وفي كل بدء للأمور ختام',
       'لم تبق عين أسعدت ذا عبرة    الا بكت حتضى بكى الحساد',
       'وكان عقيق دمعي من جفاه    من العينين يتبع شبه عين',
       'وما حسبت كفي نوالك كثرة    ولكن المحذور الردى كنت أحسب',
       'يثنى علي من العلاء خناصر    ويمد نحوي للسناء أصابع',
       'فما لياليك والأيام شاهدة    إلا تواريخ خير تكتب العجبا',
       'وذي وطن بالغور يصبو إلى الحمى    قضى وطرا منه السرى والمفاوز',
       'فدع عنك تطواع الهوى واتباعه    وشمر فقد فاز المجد المشمر',
       'كي لا يرى أثر

In [26]:
query = ['وجه كالبدر']
query_embd = model.encode(query)[0]
norm(query_embd)

21.342222

## Search

In [56]:
df_sample = df.sample(5_000, random_state=42)
df_sample

,البيت
492926,بما بعينيك من فتون ومن فتور بها وسحر
1826254,إن خطو الخطوب تقصر عمن ظل نعماك فوقه ممدود
430787,وغار منا حسدا يريد أنا نفترق
1796353,وأسخط لاستمرار هجرك ساعة وتغلب أشواقي فأرجع...
1711449,فأفاد الفواد علما خفيا ليس تحصيه في المقال ...
...,...
1659359,فاشكروا الله في على ما وهب إن شكر الرب
750584,هو البدر إلا أنه غير خائف من النقص يوما حين...
850329,سنابكها عزت بإذلالك العدا فداست خدودا كن من...
942551,لاحت الزهرة تستغوي النهى ببهاء فتنة للمبصرين


In [68]:
def similarity(verse, query):
    embeddings = model.encode([verse, query])
    return 1 - spatial.distance.cosine(embeddings[0], embeddings[1])

query = 'غيث عظيم'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
577161,وكيف نجد في استسقاء غيث لقبر ضمن الغيث الهطولا,0.640459
774078,مدى الأيام ما هطلت سماء ومد بمفعم السيل الشعاب,0.610179
1803868,أنت فأل الربى مثل غيث المطر,0.588855
685218,كل سقى قبره في حين بهاطل هتون وشيك من تدفقه...,0.583925
808869,غيث نماه سفوح وابتغائي نعمة أخرى وحسبي مبتغا,0.578670
...,...,...
1070511,ليلى اجل ليلى التي اعززتها هي كل ما انا في ...,0.139000
733444,وعلى جماعته السلام فإنهم مثل النجوم من اقتد...,0.134709
1752470,واحكم على النسبة بالفعلية وسمها مطلقة الكيفية,0.134694
535577,وإذا رمت أن تمارى فاسكت وإذا شئت أن تلاحي ف...,0.123908


In [63]:
query = 'ماء'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
1683221,وركوة الماء قد فاضت مواردها ري وطهر وباقى ا...,0.409656
873980,كأن ماء الورد قد خولطت سلساله من مائك الأطهر,0.405108
114020,وكأس قد شربت بماء ثلج وأخرى قد شربت بقاصرينا,0.396843
587581,رقت فلو مزج الماء القراح بها والراح لامتزجت...,0.384308
950381,كأنهار الفرات بعذب ماها تدل الشاربين على ال...,0.353752
...,...,...
731733,بالذكر وبالكتاب والأملاك ال عمال بكل موقع كوني,-0.021776
642324,وتحطمت عند القرون قرونهم بل كلت الأنياب وال...,-0.027530
1284951,هذه نجوم تبدو تحت النقاب أم ذا سلوس مرجان,-0.028332
1585201,وكذلك الأمجاد أسرته الألى طابوا نجارا من قد...,-0.043384


In [64]:
query = 'عيون باكية'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
1449419,تناس ما شئت تخبو حرارة الدمع في الشئون,0.652408
639142,نزفنا دموع المقلتين تفجعا فهل مرة أجدى علين...,0.645870
1391091,واستدرات شفتاها في حنان وحنين,0.626833
1788247,يضحكون ويبكون في لحظة,0.624733
1565900,ولو كان عندي غير زفرة آسف وحسرة أشعار ودمعة...,0.622477
...,...,...
1236346,يا خرية العدس الصحي ح النيء والخبز الفطير,0.121826
1708489,فإذا سمعت حديثنا بين الصفا والمنحنى فاشرب و...,0.118670
1803121,سأوجز في ذكري فضائل حزتها وأسبق بالإيجاز من...,0.117183
733444,وعلى جماعته السلام فإنهم مثل النجوم من اقتد...,0.114871


In [65]:
query = 'عقول ساهية'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
319868,على العقول التي العادات تحجبها لذاك تنكر ما...,0.506056
756387,في انف أبناء الزمان نخوه وفي عقولهم عمى ونشوه,0.503276
176672,لا تصغوا إلى أخبار قوم يصدق مينها العقل الأرك,0.493228
1784746,سكت الكلام,0.467197
799728,من خالفت أقواله أفعاله تحولت أفعاله أفعى له,0.465690
...,...,...
1272881,لا يبرأ الزوج من الصداق في مورد التحريم وال...,0.084649
1546042,أولئك وفدى جئت مستشفعا بهم وما كان حزب الله...,0.079305
1367805,فمن يكن من بينكم محسنا فإن للتاريخ إحسانه,0.072606
1663832,رب هب لي من الجلادة صبرا آتني من لدنك رب وليا,0.070202


In [66]:
query = 'جيوش فانية'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
760402,تسير أمام الجيش قبل مسيره كتائب من آرائه وجنود,0.629522
662794,أكان داعي المهالك قبل انقلاب الممالك,0.613149
1785023,وإنما تأريخ قهر,0.607880
1792569,خسرت بغداد إذ آ وتك فيها والسلام,0.591750
615847,نيرانهم ومدافعهم لا تنقذهم من صولتكم,0.586912
...,...,...
1583434,ولله أدى جميع الحقوق وحج إلى بيته واعتمر,0.041932
1272881,لا يبرأ الزوج من الصداق في مورد التحريم وال...,0.034558
1367805,فمن يكن من بينكم محسنا فإن للتاريخ إحسانه,-0.002127
733444,وعلى جماعته السلام فإنهم مثل النجوم من اقتد...,-0.002249


In [69]:
query = 'كنز ثمين'
df_sample['similarity'] = df_sample['البيت'].apply(lambda verse: similarity(verse, query))
df_sample.sort_values(by=['similarity'], ascending=False)

,البيت,similarity
1033816,ونظمت ضمائري عقدا من الجواهر,0.499884
741145,فخذ واردده قيمته ولا تتغنمن ورقه,0.488842
1438246,إذا شئت كنزا لكسرى ويجبر مولاك كسرا,0.475095
1778081,والطل كاللؤلؤ المنثور سباق,0.461031
1782745,ولكن هذا الملك سراقه كثر,0.455858
...,...,...
943074,جعل الصلاة لهم ركوعا كلها وأتى ليشرع في الس...,0.092863
1619664,وان اله الطب أرسل وحيه بآيات الهام الى الفك...,0.091335
1684561,وكم أتاهم من القرآن مزدجر وما وجدنا لآي الك...,0.085583
785641,الله الله القدير القادر الأول المحي المميت ...,0.083465
